In [11]:
print("yo")

yo


In [12]:
%pwd

'd:\\MEDICAL_CHATBOT\\Medical_ChatBot_old\\research'

In [13]:
import os
os.chdir("../")

In [14]:
import os
os.chdir("d:\\MEDICAL_CHATBOT\\Medical_ChatBot_old")

In [15]:
%pwd

'd:\\MEDICAL_CHATBOT\\Medical_ChatBot_old'

In [16]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [17]:
#Extract Data From the PDF File
def load_pdf_file(data):
    loader= DirectoryLoader(data,
                            glob="*.pdf",
                            loader_cls=PyPDFLoader)

    documents=loader.load()

    return documents

In [18]:
extracted_data=load_pdf_file(data='Data/')

In [5]:
#extracted_data

In [20]:
#Split the Data into Text Chunks
def text_split(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks=text_splitter.split_documents(extracted_data)
    return text_chunks

In [21]:
text_chunks=text_split(extracted_data)
print("Length of Text Chunks", len(text_chunks))

Length of Text Chunks 5860


In [1]:
#text_chunks

In [23]:
from langchain_huggingface import HuggingFaceEmbeddings


In [24]:
#Download the Embeddings from Hugging Face

def download_hugging_face_embeddings():
    embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings


In [ ]:
import ipywidgets as widgets
widgets.IntSlider()

In [52]:
embeddings = download_hugging_face_embeddings()

In [9]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [4]:
#query_result

In [28]:
from dotenv import load_dotenv
load_dotenv()

True

In [ ]:
import os
from openai import AzureOpenAI
    
def ai_client():
    client = AzureOpenAI(
        openai_api_base=, 
        AZURE_DEPLOYMENT_NAME='medibot',
        api_key=os.getenv(),  
        api_version="2024-11-20",
        azure_endpoint=os.getenv("")
    )

In [ ]:
import os
from openai import OpenAI
os.environ["PINECONE_API"] = ''
# os.environ["OPENAI"] = 

In [ ]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import os

pc = Pinecone(api_key='')

index_name = "medicalbot"

if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=384,
        metric="cosine",
        spec=ServerlessSpec(
            cloud="aws",
            region="us-east-1",
            )
        )
else :
    print(f"Index {index_name} already exists.")
index = pc.Index(index_name)

In [33]:
index.describe_index_stats()

{'dimension': 384,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 0}},
 'total_vector_count': 0}

In [34]:
# Embed each chunk and upsert the embeddings into your Pinecone index.
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings, 
)

In [35]:
# Load Existing index 

from langchain_pinecone import PineconeVectorStore
# Embed each chunk and upsert the embeddings into your Pinecone index.
docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)

In [3]:
#docsearch

In [37]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})

In [38]:
retrieved_docs = retriever.invoke("What is Acne?")

In [2]:
#retrieved_docs

In [ ]:
# from langchain_openai import AzureOpenAI
# import os

# # Initialize Azure OpenAI
# llm = AzureOpenAI(
#     deployment_name='gpt-4-32k',  # Correct deployment name
#     api_key=os.environ["AZURE_OPENAI_API_KEY"],  # API key from environment variable
#     azure_endpoint=,  # Correct parameter for endpoint
#     api_version='',  # Ensure the API version matches your Azure OpenAI setup
#     temperature=0.4,
#     max_tokens=500
# )

In [ ]:
# from langchain_openai import AzureChatOpenAI  # Use AzureChatOpenAI for chat models
# import os

# # Initialize Azure OpenAI with chat model
# llm = AzureChatOpenAI(
#     deployment_name='gpt-4-32k',  # Correct deployment name
#     api_key='',
#     azure_endpoint='',
#     api_version='0613',  # Ensure the API version matches your Azure OpenAI setup
#     temperature=0.4,
#     max_tokens=500
# )

# # Define the system prompt
# system_prompt = (
#     "You are an assistant for question-answering tasks. "
#     "Use the following pieces of retrieved context to answer "
#     "the question. If you don't know the answer, say that you "
#     "don't know. Use three sentences maximum and keep the "
#     "answer concise."
#     "\n\n"
#     "{context}"
# )

# # Create the prompt template
# from langchain_core.prompts import ChatPromptTemplate
# prompt = ChatPromptTemplate.from_messages(
#     [
#         ("system", system_prompt),
#         ("human", "{input}"),
#     ]
# )

# # Create the question-answering chain
# from langchain.chains.combine_documents import create_stuff_documents_chain
# question_answer_chain = create_stuff_documents_chain(llm, prompt)

# # Ensure the retriever is properly initialized
# retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k": 3})  # Replace `docsearch` with your actual retriever

# # Create the retrieval chain
# from langchain.chains import create_retrieval_chain
# rag_chain = create_retrieval_chain(retriever, question_answer_chain)

# # Invoke the chain with a question


In [ ]:
# from langchain.chat_models import AzureChatOpenAI

# llm = AzureChatOpenAI(
#     deployment_name='gpt-4',
#     model_name = "gpt-4",  # Correct deployment name
#     api_key='',
#     azure_endpoint=',
#     api_version='',  # Ensure the API version matches your Azure OpenAI setup
#     temperature=1.0,
#     max_tokens=4096
    
# )

# from langchain_openai import OpenAI

# llm = OpenAI(
#     api_key='',
#     temperature=0.4,
#     max_tokens=500
# )

# from openai import AzureOpenAI
# client = AzureOpenAI(
#     api_version='turbo-2024-04-09',
#     azure_endpoint='',
#     credential=""
# )

In [40]:
from langchain_openai import OpenAI
llm = OpenAI(temperature=0.4, max_tokens=500)

In [41]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate


system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)


prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [42]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [ ]:
response = rag_chain.invoke({"input": "what is Acromegaly and gigantism?"})
print(response["answer"])

In [ ]:
# try:
#     response = rag_chain.invoke({"input": "what is Acromegaly and gigantism?"})
#     print(response["answer"])
# except Exception as e:
#     print(f"Error: {e}")

In [ ]:
response = rag_chain.invoke({"input": "What is stats?"})
print(response["answer"])